# Tranformers

In the previous chapter, we covered RNNs, the modeling architecture in vogue in NLP until the Transformer architecture gained prominence.

Transformers are the workhorse of modern NLP. The original architecture, first propsed in 2017, has taken the (deep learning) world by storm. Since then, NLP literature has been inundated with all sorts of new architectures that are broadly classified into either Sesame street characters or words that end with "-former"

In this chapter, we'll look at that very architecture-the transformer-in detail. We'll analyze the core innovations and explore a hot new category of neural network layers: the attention mechanism.

## Building a Transformer from Scratch

In Chapter 2 and 3, we explored how to use transformers in practice and how to leverage pretrained transformers to solve complex NLP problems. Now we're going to take a deep dive into the architecture itself and learn how transformers work from first principles.

What does "first principles" mean? Well, for starters, it means we're not allowed to use the Hugging Face Transformers library. We've raved about it plenty in this book already, so it's about time we take a break from that and see how things actually work under the hood. For this chapter, we're going to be using raw PyTorch instead.

PyTorch, being a fully fledged deep learning library that most researchers use, naturally has an implementation of the extremly popular transformer architecture, just like a Hugging Face library does. This version, though, exposed as an nn.Module, is much more DIY and is meant to be used with the other familiar PyTorch tools like dataloaders, optimizers, etc.

As we've mentioned before, one of the best ways to see what any deep learning related class/function does is by looking at the type signature and he dimensionality of the inputs and outputs. So let's do that:

In [ ]:
import torch

In [ ]:
model = torch.nn.Transformer()
model.encoder.layers[0]

In [ ]:
TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
        (out_proj): Linear(in_features=512, out_features=512, bias=True)
    )
    (linear1): Linear(in_features=512, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=512, bias=True)
    (norm1): LayerNorm(512,), eps=1e-0.5, elementwise_affine=True)
    (norm2): LayerNorm(512,), eps=1e-0.5, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
)    

At the outset, there doesn't seem to be too much to take away from this. It's a fairly standard PyTorch nn.Module with the standard forward() function defined for us. In principle, we could just plug it into our training pipeline and carry on. in fact, this is essentially what we did in Chapter 2. But let's try to understand the exact components of this module.

Of particular interest is the MultiheadAttention layer. Most of the other layers, like Dropout, Linear, and LayerNorm, are things you'd expect to see in nontransformer models as well.This particular implementation of he transformer by PyTorch (with no additional configuration parameters), exactly matches the specification of the architecture in the original paper (shown in Figure 7-1) which, coincidentally, is titled "Attention Is All you Need."

In short, it's safe to say that the most important component of this Transformer class is the MultiheadAttention layer. So it makes sense to take some time to understand what that is and how it works.

## Attention Mechanisms

An attention mechanism is a layer in a deep neural network. Its job, while still open to interpretation, is to learn long-range, "global" features. An attention mechanism acts as what we like to call an "information router" that decides what components of the input sequence of embedding vectors contribute to a single output vector. This idea will become more clear as we actually work through the details.

We're just as excited to talk about attention as the other couple thousand people that attended NeurIPS within that last year, but before we de, we should mention that an important theme to pay attention to is the computaional complexity of the operations involved.Think about how many dot products/matrix multiplications you see and the size of the tensors involved.

### Dot Product Attention

OK, strictly speaking, we don't think we've seen this type of attention actually being applied in real networks. Scaled dot product attention is usually just talked about as a component of the next thing we'll discuss: Multi-Head Self-Attention.

The most important question you need to ask in the world of exotic attention mechanisms is this: how, exactly, do you measure the similarity between things? This core idea, shrouded in a veil of linear algebra and bucket-loads of GPUs, is what drives the fundamental behavior of neural nets in NLP today.

And the scaled dot product uses probably one of the simplest and most intuitive methods of measureing similarity-the dot product.

You should be familiar with this, but let's do a quick recap. The dot product is an operation that takes two vectors, multiples them element-wise, and then adds up the results. This measures similarity becasue if the two vectors that we're "dot-producting" have similar components, the product of their elements will be large, and vice versa (in the sense that vectors with dissimilar components will have a small dot product).

But the real question is, what exactly are we taking the dot product of? To answer this question, let's focus on how these attention mechanisms are implemented in transformers(see Figure 7-1).

![image](images/transformer_layer.png)

A typical transformer takes in sequences of word vectors as input, and at each layer, transforms (and no, we don't think that's how they got their name) them into another sequence of vectors, which we call the hidden representation/state.

So at each hidden layer in the network, we have sequences of vectors that we want to "attend" over. See Figure 7-2.

![image](images/dotproduct_1-768x412.png)

Now, here's the important bit, so pay attention (pun very much intented).

What we're going to do is transform each one of these hidden state vectors into three seperate, completely independent vectors-the query, the key, and the value.

We do this transformation via a simple matrix multiply, and the dimensions of these vectors are up to us. The only restriction is that the query and key vectors need to have the same dimensions (since we're going to take the dot product between them):

We then compute the attention weights by taking the dot product of the query vector at each time step as with all of the key vectors, and softmax the result. To do this over all time steps simultaneously, it's more efficient to pack these vectors into a matrix taht performs the multiplicaitons in parallel. The final calculation would look something like this:

That's not all we can do, though. Since each of the query vectors are independent, we can parallelize across time:

But why? That's the question we were asking ourselves when reading the transformer paper. Splitting into three vectors seems a little arbirary and complicated. Like, why not two or four?

Base on just the naming, it seems like the intuition here is rooted in databases.Think about how this would work in regular old Python dictionaries, no neural nets involved.

You have a large sequence of key-value pairs. That's the dictionary structure. I could look something like this:

```
sentence = {
    "word_1": "Squirtle"
    "word_2": "is"
    "word_3": "the"
    "word_4": "greatest"
    "word_5": "Pokemon"
    "word_6": "ever"
}
```

Now I know what you're thinking-"Puh-lease! We all know Squirtle doesn't stand a sliver of a chance against Charizard."

Well, We beg to differ. Deal with it.

When you want to get a value from the dictionary, you'd use a query that looks something like this:

```
a = sentence['word_3']
```

And what Python would do, behind the scenes, is compare your query word_3 against all the possible keys in the sentence dictionary. It would then return the value and store it in a vairable.

What we're doing with dot product attention is similar. The query vector represents, in some abstract sense, what the current word is looking for. The key associated with each word kind of represents what each word has to offer. The value vector contains the information that the query vector was looking for. But we know that sounds super abastract, so let us show you an example.

Consider the following sentence:

>Mario is short, but the he can jump super high.

Now say our transformer is currently working on the word "he," and it's trying to propagatge it to the next layer in the network. The query vector here might be something that's looking for a name or a person to clarify what exactly the pronoun "he" is referring to. So, the transformer takes the query vector for "he" and computes dot products of this query vector with the key vectors of every other word in the sentence. Each of these dot products generates a sort of alignment socre that measures how much the query and key match.

As it does it, the key vector corresponding to "Mario" is likely to light up, and will generate the largest alignment score. This indicates to the network that there's something interesting going on there, and the network should pay attention (see what we did there?).

But the job is'net done yet. Once the transformer calculates all the alignment socres between "he" and the other words in the sentence, it passes these scores through a softmax, to generate a nice distribution. You can interpret the score more naturally: 0 would tell you that there's little connection between the words, and a 1 would tell you that there's a near-perfect alignment.

Remember that each word also has an associated value vector, which, in our picture, is supposed to represent tha actually meaningful connect of the world, just like in the case of values in Python dictionaries. Unlike Python dictionaries, however, each query doesn't return a single result. Instead, the transformer takes the normalized alignment scores that we calcuated for each word and uses them to perform a wighted sum over all athe value vectors. The reason for this is somewhat simple-say the jump very high." Here, the query for "they" is not just looking for a single word, but every possible person that fits into this group.

Creating a distribution of alignment scores now lets us pick up different parts of the sentence in different amounts! Oversimplifying a bit, you can imagine that the normalzied alignement scores for the words "Mario" and "Luigi" are 0.5 and 0 for all other words.

The transformer has now attented (seriously, is that even a word?) over the sentence, and has created a vector for a particular word ("he" and "they", in our example) that encapsulates how the relevant parts of the sentence related to this word in the grand shceme of things.

You'd now repeat this process for every word in the sentence, thereby getting another sequence of vectors to pass on to deeper layers in the network.

When we're computing the so-called "self-attenction" in the encoder parts of the transformer, the following hidden states are used to calculate:

These all come from the sequence in that layer of the encoder. The same is true for self-attention in the decoder.

There's another one, though-the attention layer used in the decoder that uses the decoder hidden representation for the queries, and the encoder hidden representation for the keys and values. This allows the decoder to attend over all previous encoder hidden representations, which is useful in tasks like machine translation. You wouldn't want you French translator to starts spewing out gibberish without actually reading the whole sentence in English first.

You can visualize the self-attention layer in Figure 7-2 and the entire layer put together in Figure 7-1. Stack a few of these layers on top of one another, and boom! You've (almost) got yourself a transformer.

### Scaled Dot Product Attention

There's a minor problem with this, though. Although dot products are really fast, cool, and all that, when the size of the vectors are large, the dot product can get pretty big.

To see what we mean, consider two random vectors. Instead of just talking about it, though, let us show you some actual computations from NumPy:

In [ ]:
import numpy as np

In [ ]:
small_dots = [
    np.dot(np.random.randn(10),
           np.random.randn(10))
    for i in range(100)]
np.mean(np.absolute(small_dots))

What we just did there was generate two random arrays of size 10 and take the dot product of them. Just to be sure, we repeated this 100 times and calculated the average magnitude of the dot product to make sure that we're not getting a random outlier.

And so what the value was around 2.74. How's that usefull? Well, let's try the same thing with arrays of size 10,000:

In [ ]:
large_dots = [np.dot(np.random.randn(10000),
                     np.random.randn(10000))
              for i in range(100)]
np.mean(np.absolute(large_dots))

Ok. That's a lot bigger. But think about it-since we're using dot products to measure alignment, something is clearly wrong here. In both cases, we generated purely random vectors, so ideally, their alignment scores should be similar.

But since the components are chosen from a standard normal distribution with mean 0 and variance 1, an n-dimensional vector will have a variance of n (you get his by adding up the variances of the components, and if you're going to be pedantic, it's the trace of the covariance matrix of the vector, but that's way too long a name).

To correct for this and ensure that vectors of any dimensionality have roughly the same alignment scores, we'll scale our previous attention mechanism similar to how you'd normalize to unit variance in statistics.

The new, corrected attention mechanism would be:

### Multi Head Self Attention

Here's something you might find interesting: the two attention mechanisms that we just discussed, and the one we're about to show you now, all came from the same paper-"Attention Is All You Need" (aka the transformer paper). Pretty cool, huh?

Anyway, the next thing we can do is try to split up our attention mechanism into many smaller attention mechanisms (with an "s"). Why would wnat to do this? A good way to illustrate the rationale is through a popular attention test video (https://youtu.be/vJG698U2Mvo)

Now you've probably seen that video before (if you haven't, surprise!), and you know why it's so hard to spot the gorilla on your first pass-it's easier and more national to pay attention to one thing at a time. In this case, that's baskeball passes, since that's what the video asks you to look for. If instead, you were asked to look for a gorilla, it probably would have been easier to find the gorilla.

Attention mechnisms kind of work in the same way. There's a lot of stuff to pay attention to and keep track of in language, like pronouns, as we discussed earlier ("Mario is short, but he can jump super high"), but also other things, like where the main characters are going in physical space ("Mario went to the flower store and then to the gym, where he did 50 squats")

Having one set of queries, keys, and values do all that work might be a bit too much, and they might miss out on the occational gorilla, just like you probably did.

Multi-head attention mechanisms try to fix this issue by independently applying the attention mechanism multiple times on the same sequence in a single pass. In terms of the gorilla video, this would be like having your buddy watch the video with you. One of you could pay attention to the passes, while the other could look for gorillas, thereby increasing the overall attention capablilities.

Crucially, the query key and value matrices need to be different, otherwise redoing the whole attention thing multiple times would just be a waste of computation (asking your friend to look for passes while you also look for passes).

To create variety in the queries, keys, and values, the transformer network simply uses multiple sperate weight matrices to transform the input into multiple queiries, keys, and values:

Here, n is the parameter you set, and it's called the number of heads. It represents how many different attention computations are being done on the same sequence. You can think of it as the number of people you invite over to watch that gorilla video with you.

Are you tired of that analogy yet? Don't worry, we're almost done with it.

Each one of these "heads" performs the scaled dot product attention calculation independently (and, crucially, in parallel):

At the end of all this number-crunching, we're going to be left n different output vectors per spot in the sequence, corresponding to the outputs from each of the attention heads. But since the next layer needs a sequence of vectors (and a sequence of n vectors), the transformer concatenates the output from the multiple attention heads and passes it through another learned linear transform to make the dimenstions work right:

A sequence of these new concatenated and transformed z vectors is what gets passed on to the next layer of the transformer.

That's difinitely a lot of linear algebra to take in at once, so go through it slowly again to make sure you actually get it. In particular, visualizing Multi-Head Self-Attention is probably the best way to understand how it works. Jay Alammar has an exellent set of article on this (https://jalammar.github.io/illustrated-gpt2/), and we hightly encourage you to take a look at the viusalizations presented there.

### Adaptive Attention Span

OK, we're finally moving on to some (relatively) newer and cooler stuff. In 2019, some cool people at Facebook AI Research asked a really cool question-what if we could get transformer to learn what to pay attention to?

But isn't that what transformers aleardy do? Isn't this the entire point of the attention mechanism?

Well, yes. But there's also another very important thing we haven't talked about-computaional cost. You see, adding an attention mechanism isn't cheap. If you have n words in a batch/sentence, it would taken $n^2$ dot products (per layer) to compute each of the attention weights across all the tokens in the sequence. This is because you have to take the dot product of each of the $n$ query vectors with each of the n keys.

As you can tell, this can blow up pretty fast. If you had, say, 50 takens/words in your batch/sentence, then there are at least $50^2 = 2,500$ dot products to compute. But simply by increasing the number of tokens by two, to 52, you'd now have more than $52^2 = 2704$ dot products to compute. That's about 200 more dot products justfor adding two extra tokens per batch, and that's not even factoring in multi-headed attention!

Of course, one could question if we really need to compute attention over every single token every single time. It sees a little excessive. Especially in character-level or subword-level models, where some of the attention heads might simply be looking at the last few tokens to try and fit characters or subwords together into words. But then make every head look at only the last few tokens.

The way we (or in this case, the Facebook team; we are just reaping the benefits of their work) strike a balance is by having some heads attend over a larger set of tokens, and have some heads attend over only the last few tokens.

There's one term we'll introduce here: attend span. This simply refers to how many previous tokens the model is attending to. So if a head has an attention span of 5, this means that head runs an attention mechaniszm over the last 5 tokens from the current position in the sequence.

So how do we decide the attention span for each of the heads? Typically, this would involve experiments, plots, some hand-waving, and a fair bit of guesswork. But what makes adaptive attention span so cool is that each head can learn its own attention span though the training process!

This idea is really cool because it takes something that would have been a hyperparameter, the number of tokens to attend over, and makes it a simple parameter that can be automatically tuned through backprop.

Here's the main issue at hand: the number of tokens that each head looks at, also called the attention space, is an interger, and therefore can't be differentiated. Being nodifferentiable means that you can't really learn that parameter though training. So instead, the research team had to come up with a clever way to get a differentiable version of the attention span.

They did this by creating something called a masking function, which takes in th distance between tokens and outputs a value between 0 and 1. In the paper, they define the masking function like this:

Which we guess looks a little weird. But the plot is actually pretty clear and simple, as shown in Figure 7-3.

So the intuition here is that if the distance x between two tokens is large enough, the value of $m_z(x)$ will be zero, which means we don't do the attention computation between those two tokens.

Since this $m_z(x)$ fuction is smooth, we can get its gradient and tune the value of $z$ for each attention head. With a larger $z$, the attention head would look across more tokens, and vice versa. $R$ is a hyperparamter that controls the smoothness of that ramp section you see on the plot.

But most importantly, the adaptive attention span transformer has some pretty cool results. It achives state-of-the-art performance on the enwik8 dataset using considerably less memory and FLOPs than other transformers.

### Persistent Memory/All-Attention

### Product-Key Memory

## Transformers for Computer Vision

# Fine-tuning BERT for sentiment analysis

## Contents
1. 학습데이터 확인
2. 보조함수 확인
3. SenitmentClassifier 확인
4. Analyser 확인
5. 학습 과정 확인
6. tests

## 학습 데이터

In [ ]:
!pip3 install torch
!pip3 install transformers
from typing import List, Tuple
from transformers import BertTokenizer, BertModel
import torch
from torch.nn import functional as F


DATA: List[Tuple[str, int]] = [
    # 긍정적인 문장 - 1
    ("난 너를 좋아해", 1),
    # --- 부정적인 문장 - 레이블 = 0
    ("난 너를 싫어해", 0)
]

TESTS = [
    "나는 자연어처리가 좋아",
    "나는 자연어처리가 싫어",
    "나는 너가 좋다",
    "너는 참 좋다",
]

## 보조함수

In [ ]:
# cuda를 사용할 수 있는지를 체크, 사용가능하다면 cuda로 설정된 device를 출력.
def load_device() -> torch.device:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return device


# 텐서를 구축하는 부분 - X
def build_X(sents: List[str], tokenizer: BertTokenizer, device: torch.device) -> torch.Tensor:
    """
    return X (N, 3, L).  N, 0, L -> input_ids / N, 1, L -> token_type_ids / N, 2, L -> attention_mask
    """
    encodings = tokenizer(text=sents,
                          add_special_tokens=True,
                          return_tensors='pt',
                          truncation=True,
                          padding=True)
    return torch.stack([
        encodings['input_ids'],
        encodings['token_type_ids'],
        encodings['attention_mask']
    ], dim=1).to(device)

# 텐서를 구축하는 부분 - y
def build_y(labels: List[int], device: torch.device) -> torch.Tensor:
    return torch.FloatTensor(labels).unsqueeze(-1).to(device)

## Sentiment Classifier

In [ ]:
class SentimentClassifier(torch.nn.Module):
    def __init__(self, bert: BertModel):
        super().__init__()
        self.bert = bert
        self.hidden_size = bert.config.hidden_size
        # TODO 1
        self.W_hy = torch.nn.Linear(..., ...)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        :param X: (N, 3, L)
        :return: H_all (N, L, H)
        """
        input_ids = X[:, 0]
        token_type_ids = X[:, 1]
        attention_mask = X[:, 2]
        H_all = self.bert(input_ids, token_type_ids, attention_mask)[0]
        return H_all

    def predict(self, X: torch.Tensor) -> torch.Tensor:
        """
        :param X:
        :return:
        """
        # TODO 2
        H_all = self.forward(X)
        H_cls = ...
        y_hat = ...  # (N, H) * (H, 1) -> (N, 1)
        return torch.sigmoid(y_hat)

    def training_step(self, X: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        # TODO 3
        y_hat = self.predict(X)
        loss = ...
        return loss.sum()

## Analyser

In [ ]:

class Analyser:
    """
    Bert 기반 감성분석기.
    """
    def __init__(self, classifier: SentimentClassifier, tokenizer: BertTokenizer, device: torch.device):
        self.classifier = classifier
        self.tokenizer = tokenizer
        self.device = device

    def __call__(self, text: str) -> float:
        X = build_X(sents=[text], tokenizer=self.tokenizer, device=self.device)
        y_hat = self.classifier.predict(X)
        return y_hat.item()

## Training

In [ ]:
# 사전학습된 버트 모델을 로드
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
bert = BertModel.from_pretrained('beomi/kcbert-base')

# --- have a look at the config --- #
print(bert.config)
print(bert.config.hidden_size)

In [ ]:
# --- hyper parameters --- #
EPOCHS = 20
LR = 0.0001


device = load_device()
print(device)

# --- build the dataset --- # 
sents = [sent for sent, _ in DATA]
labels = [label for _, label in DATA]
X = build_X(sents, tokenizer, device)
y = build_y(labels, device)

# --- instantiate the classifier --- #
classifier = SentimentClassifier(bert)
classifier.to(device)  # 모델도 gpu에 올리기. 
optimizer = torch.optim.Adam(classifier.parameters(), lr=LR)  # 최적화 알고리즘을 선택.

# --- 학습시작 --- #
for epoch in range(EPOCHS):
    loss = classifier.training_step(X, y)
    loss.backward()  # 오차 역전파
    optimizer.step()  # 경사도 하강
    optimizer.zero_grad()  # 기울기 축적방지
    print(f"epoch:{epoch}, loss:{loss.item()} ")

## Test

In [ ]:
classifier.eval()
analyser = Analyser(classifier, tokenizer, device)

for sent in TESTS:
    print(sent, "->", analyser(sent))

## Conclusion